In [1]:
import pandas as pd
import numpy as np
import os
import math
from itertools import product

In [3]:
df = pd.read_csv(os.path.join(os.path.expanduser('~/influence_score_pruning_sexism/score_results'), 'sexist_data_final_train.csv'))
df.head()

,id,text,numeric_labels,sexist,predicted_label_1,correct_yx,predicted_label_2,predicted_label_3,predicted_label_4,predicted_label_5,average_pvi,average_el2n,average_vog,misclassification_number
0,2651,SIIIIGH http://t.co/BpJX5JwhlH,0,False,LABEL_0,True,LABEL_0,LABEL_0,LABEL_0,LABEL_0,0.148039,0.006603,-0.832910,0
1,6089,I really dislike working with undergrads in la...,0,False,LABEL_0,True,LABEL_0,LABEL_0,LABEL_0,LABEL_0,0.132709,0.021481,-0.832839,0
2,9786,"So calling qualified hires ""tokens"" b/c it's n...",0,False,LABEL_0,True,LABEL_0,LABEL_0,LABEL_0,LABEL_0,0.142481,0.012016,-0.831354,0
3,10635,Noooooo #mkr whyyyyy,0,False,LABEL_0,True,LABEL_0,LABEL_0,LABEL_0,LABEL_0,0.145224,0.009347,-0.834254,0
4,9749,I don't know ANY genuine female nerds... who w...,0,False,LABEL_1,False,LABEL_1,LABEL_1,LABEL_1,LABEL_1,-1.236664,0.868590,-0.832807,5


In [4]:
def get_proportional_prune(data, inf_score, sexist_type_prune, non_sexist_type_prune, prune_rate, data_name = 'cmsb'):
    data_sexist = data[data['numeric_labels'] == 1].reset_index(drop = True)
    data_non_sexist = data[data['numeric_labels'] == 0].reset_index(drop = True)
    prune_rate_sexist = prune_rate / 2
    prune_rate_non_sexist = prune_rate / 2
    data_sexist_pruned, data_non_sexist_pruned = None, None
    if sexist_type_prune == 'hard' and non_sexist_type_prune == 'hard':
        data_sexist_pruned = data_sexist.iloc[-
            (
                math.ceil((
                         1 - (prune_rate_sexist / 100)) * data_sexist.shape[0])): -1].reset_index(drop = True)
        data_non_sexist_pruned = data_non_sexist.iloc[-(
            math.ceil((
             1 - (prune_rate_non_sexist / 100)) * data_non_sexist.shape[0])): -1].reset_index(drop = True)
    elif sexist_type_prune == 'hard' and non_sexist_type_prune == 'easy':
        data_sexist_pruned = data_sexist.iloc[-(
            math.ceil((
             1 - (prune_rate_sexist / 100)) * data_sexist.shape[0])): -1].reset_index(drop = True)
        data_non_sexist_pruned = data_non_sexist.iloc[
            : math.ceil((1 - (prune_rate_non_sexist/ 100)) * data_non_sexist.shape[0])].reset_index(drop = True)
    elif sexist_type_prune == 'easy' and non_sexist_type_prune == 'hard':
        data_sexist_pruned = data_sexist.iloc[
            : math.ceil((1 - (prune_rate_non_sexist/ 100)) * data_sexist.shape[0])].reset_index(drop = True)
        data_non_sexist_pruned = data_non_sexist.iloc[-
            (
                math.ceil((
                         1 - (prune_rate_non_sexist / 100)) * data_non_sexist.shape[0])): -1].reset_index(drop = True)
    elif sexist_type_prune == 'easy' and non_sexist_type_prune == 'easy':
        data_sexist_pruned = data_sexist.iloc[
            : math.ceil((1 - (prune_rate_non_sexist/ 100)) * data_sexist.shape[0])].reset_index(drop = True)
        
        data_non_sexist_pruned = data_non_sexist.iloc[
            : math.ceil((1 - (prune_rate_non_sexist/ 100)) * data_non_sexist.shape[0])].reset_index(drop = True)
    data_combined = pd.concat([data_sexist_pruned, data_non_sexist_pruned], axis = 0)
    data_combined.to_csv(os.path.join(f'{sexist_type_prune}_{non_sexist_type_prune}', f'sexist_data_proportional_{inf_score}_{prune_rate}_train.csv'))

In [5]:
df_pvi = df[['text', 'numeric_labels', 'average_pvi']].sort_values(by = 'average_pvi').reset_index(drop = True)
df_el2n = df[[ 'text', 'numeric_labels', 'average_el2n']].sort_values(by = 'average_el2n',
                                                                        ascending = False).reset_index(drop = True)
#df_vog = df[['text', 'numeric_labels', 'average_vog']].sort_values(by = 'average_vog',
#                                                                        ascending = False).reset_index(drop = True)

In [14]:
prune_types = ["easy", "hard"]
for inf_score in ["pvi", "el2n"]:
    for sexist_type_prune, non_sexist_type_prune in list(product(prune_types, prune_types)):
        for prune_rate in (5, 10, 15, 20, 25, 30, 35, 40, 50, 60):
            get_proportional_prune(df_pvi, inf_score = inf_score, sexist_type_prune = sexist_type_prune, 
                               non_sexist_type_prune = non_sexist_type_prune, prune_rate = prune_rate)

## Analysis of Datapoints

In [8]:
df_sexist_pvi = df_pvi[df_pvi['numeric_labels'] == 1].reset_index(drop = True)
#Saving top 10 hard and easy examples
df_sexist_pvi.head(10).to_csv(os.path.join('data_to_analyze','hard_examples_sexist_pvi.csv'), index = False)
df_sexist_pvi.tail(10).to_csv(os.path.join('data_to_analyze','easy_examples_sexist_pvi.csv'), index = False)

In [10]:
df_sexist_el2n = df_el2n[df_el2n['numeric_labels'] == 1].reset_index(drop = True)
df_sexist_el2n.head(10).to_csv(os.path.join('data_to_analyze', 'hard_examples_sexist_el2n.csv'), index = False)
df_sexist_el2n.tail(10).to_csv(os.path.join('data_to_analyze', 'easy_examples_sexist_el2n.csv'), index = False)

In [11]:
df_sexist_vog = df_vog[df_vog['numeric_labels'] == 1].reset_index(drop = True)
df_sexist_vog.head(10).to_csv(os.path.join('data_to_analyze', 'hard_examples_sexist_vog.csv'), index = False)
df_sexist_vog.tail(10).to_csv(os.path.join('data_to_analyze', 'easy_examples_sexist_vog.csv'), index = False)

In [27]:
from itertools import product

In [31]:
list(product(['hard', 'easy'], ['easy', 'hard']))

[('hard', 'easy'), ('hard', 'hard'), ('easy', 'easy'), ('easy', 'hard')]

In [32]:
df_non_sexist_pvi = df_pvi[df_pvi['numeric_labels'] == 0].reset_index(drop = True)
df_non_sexist_pvi.head(10).to_csv(os.path.join('data_to_analyze','hard_examples_non_sexist_pvi.csv'), index = False)
df_non_sexist_pvi.tail(10).to_csv(os.path.join('data_to_analyze','easy_examples_non_sexist_pvi.csv'), index = False)

In [33]:
df_non_sexist_el2n = df_el2n[df_el2n['numeric_labels'] == 0].reset_index(drop = True)
df_non_sexist_el2n.head(10).to_csv(os.path.join('data_to_analyze', 'hard_examples_non_sexist_el2n.csv'), index = False)
df_non_sexist_el2n.tail(10).to_csv(os.path.join('data_to_analyze', 'easy_examples_non_sexist_el2n.csv'), index = False)
df_non_sexist_vog = df_vog[df_vog['numeric_labels'] == 0].reset_index(drop = True)
df_non_sexist_vog.head(10).to_csv(os.path.join('data_to_analyze', 'hard_examples_non_sexist_vog.csv'), index = False)
df_non_sexist_vog.tail(10).to_csv(os.path.join('data_to_analyze', 'easy_examples_non_sexist_vog.csv'), index = False)